<a href="https://colab.research.google.com/github/dakyommii/study/blob/main/yolo/trafficCustomYoloDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Preparation

Runtime => GPU

In [ ]:
#dataset download
!wget https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/FullIJCNN2013.zip

--2021-03-27 15:43:18--  https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/FullIJCNN2013.zip
Resolving sid.erda.dk (sid.erda.dk)... 130.225.104.13
Connecting to sid.erda.dk (sid.erda.dk)|130.225.104.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1661866983 (1.5G) [application/zip]
Saving to: ‘FullIJCNN2013.zip’

FullIJCNN2013.zip   100%[===================>]   1.55G  25.1MB/s    in 64s     

2021-03-27 15:44:23 (24.6 MB/s) - ‘FullIJCNN2013.zip’ saved [1661866983/1661866983]



In [ ]:
!unzip FullIJCNN2013.zip

Archive:  FullIJCNN2013.zip
   creating: FullIJCNN2013/
   creating: FullIJCNN2013/00/
  inflating: FullIJCNN2013/00000.ppm  
  inflating: FullIJCNN2013/00001.ppm  
  inflating: FullIJCNN2013/00002.ppm  
  inflating: FullIJCNN2013/00003.ppm  
  inflating: FullIJCNN2013/00004.ppm  
  inflating: FullIJCNN2013/00005.ppm  
  inflating: FullIJCNN2013/00006.ppm  
  inflating: FullIJCNN2013/00007.ppm  
  inflating: FullIJCNN2013/00008.ppm  
  inflating: FullIJCNN2013/00009.ppm  
  inflating: FullIJCNN2013/00010.ppm  
  inflating: FullIJCNN2013/00011.ppm  
  inflating: FullIJCNN2013/00012.ppm  
  inflating: FullIJCNN2013/00013.ppm  
  inflating: FullIJCNN2013/00014.ppm  
  inflating: FullIJCNN2013/00015.ppm  
  inflating: FullIJCNN2013/00016.ppm  
  inflating: FullIJCNN2013/00017.ppm  
  inflating: FullIJCNN2013/00018.ppm  
  inflating: FullIJCNN2013/00019.ppm  
  inflating: FullIJCNN2013/00020.ppm  
  inflating: FullIJCNN2013/00021.ppm  
  inflating: FullIJCNN2013/00022.ppm  
  inflating: Ful

In [ ]:
#annotation file
!ls -al ./FullIJCNN2013/gt.txt 

-rw-r--r-- 1 root root 35282 Jul  9  2013 ./FullIJCNN2013/gt.txt


In [ ]:
import pandas as pd
import os
import cv2

In [ ]:
test_percentage = 0.2
file_name = './FullIJCNN2013/gt.txt'

In [ ]:
#반드시 이 형식을 지켜야 데이터를 잘 뽑아올 수 있음
annotation = pd.read_csv(file_name,
                         names=['ImageId', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2', 'ClassId'],  #coloumn names 지정
                         sep=';') #각 파일이 세미콜론으로 seperate되어 있기 때문에 seperate 명시 필요

In [ ]:
annotation.head()

,ImageId,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId
0,00000.ppm,774,411,815,446,11
1,00001.ppm,983,388,1024,432,40
2,00001.ppm,386,494,442,552,38
3,00001.ppm,973,335,1031,390,13
4,00002.ppm,892,476,1006,592,39


In [ ]:
#4가지 카테고리로 나누기
Prohibitory = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 15, 16]
Danger = [11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
Mandatory = [33, 34, 35, 36, 37, 38, 39, 40]
Other = [6, 12, 13, 14, 17, 32, 41, 42]

categories = ['Prohibitory', 'Danger', 'Mandatory', 'Other']

In [ ]:
annotation['CategoryId'] = 0  #'CategoryId' 라는 새로운 column 생성해 초기값 지정 

In [ ]:
#카테고리 별로 서로 다른 CategoryId 부여
annotation.loc[annotation['ClassId'].isin(Prohibitory), 'CategoryId'] = 0
annotation.loc[annotation['ClassId'].isin(Danger), 'CategoryId'] = 1
annotation.loc[annotation['ClassId'].isin(Mandatory), 'CategoryId'] = 2
annotation.loc[annotation['ClassId'].isin(Other), 'CategoryId'] = 3

In [ ]:
#절대적인 centerX, centerY, width, height 구하고 각각 칼럼으로 만들어 추가 
annotation['centerX'] = (annotation['Roi.X2']+annotation['Roi.X1']) / 2 
annotation['centerY'] = (annotation['Roi.Y2']+annotation['Roi.Y1']) / 2 
annotation['width'] = (annotation['Roi.X2'] - annotation['Roi.X1']) 
annotation['height'] = (annotation['Roi.Y2'] - annotation['Roi.Y1']) 

In [ ]:
annotation.head()

,ImageId,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,CategoryId,centerX,centerY,width,height
0,00000.ppm,774,411,815,446,11,0,794.5,428.5,41,35
1,00001.ppm,983,388,1024,432,40,0,1003.5,410.0,41,44
2,00001.ppm,386,494,442,552,38,0,414.0,523.0,56,58
3,00001.ppm,973,335,1031,390,13,0,1002.0,362.5,58,55
4,00002.ppm,892,476,1006,592,39,0,949.0,534.0,114,116


In [ ]:
image_path = './FullIJCNN2013'

In [ ]:
os.chdir(image_path)

In [ ]:
!ls -al

In [ ]:
paths = []
for current_dir, dirs, files in os.walk('.'):
    for file in files:
        if file.endswith('.ppm'): #dataset의 모든 ppm file 가져오기 
            image_ppm = cv2.imread(file)
            h, w = image_ppm.shape[:2]  #이미지 크기 측정
            image_name = file.split('.')[0] #이미지 이름만 가져오기
            #파일 형식 지정
            txt_file = image_name + '.txt'  
            image_jpg = image_name + '.jpg'
            cv2.imwrite(image_jpg, image_ppm) #ppm file을 jpg file로 바꿔주기
            txt = open(txt_file, 'w')
            objects = annotation.loc[annotation['ImageId'] == file].copy()  #imageId가 file 명과 같아야함 
            for index, data in objects.iterrows():
                line = str(data['CategoryId']) + ' ' + str(data['centerX']/w) + ' ' + str(data['centerY']/h) + ' ' + str(data['width']/w) + ' ' + str(data['height']/h) + '\n'
                txt.write(line) 
            image_path = image_jpg + '\n'
            paths.append(image_path)
            print(image_jpg, ' is crated')             

00035.jpg  is crated
00111.jpg  is crated
00893.jpg  is crated
00643.jpg  is crated
00181.jpg  is crated
00607.jpg  is crated
00695.jpg  is crated
00339.jpg  is crated
00201.jpg  is crated
00383.jpg  is crated
00050.jpg  is crated
00089.jpg  is crated
00673.jpg  is crated
00516.jpg  is crated
00193.jpg  is crated
00509.jpg  is crated
00373.jpg  is crated
00416.jpg  is crated
00254.jpg  is crated
00863.jpg  is crated
00005.jpg  is crated
00144.jpg  is crated
00014.jpg  is crated
00309.jpg  is crated
00716.jpg  is crated
00331.jpg  is crated
00324.jpg  is crated
00683.jpg  is crated
00624.jpg  is crated
00277.jpg  is crated
00444.jpg  is crated
00192.jpg  is crated
00880.jpg  is crated
00661.jpg  is crated
00847.jpg  is crated
00656.jpg  is crated
00559.jpg  is crated
00076.jpg  is crated
00885.jpg  is crated
00871.jpg  is crated
00828.jpg  is crated
00134.jpg  is crated
00311.jpg  is crated
00500.jpg  is crated
00227.jpg  is crated
00274.jpg  is crated
00301.jpg  is crated
00646.jpg  is

In [ ]:
#txt, jpg, ppm이 쌍으로 형성된 것을 확인할 수 있음
!ls -al

total 3183276
drwxr-xr-x 45 root root   73728 Mar 27 16:02 .
drwxr-xr-x  1 root root    4096 Mar 27 15:44 ..
drwxr-xr-x  2 root root    4096 Jul  9  2013 00
-rw-r--r--  1 root root  361627 Mar 27 16:03 00000.jpg
-rw-r--r--  1 root root 3264016 Nov 26  2012 00000.ppm
-rw-r--r--  1 root root       0 Mar 27 16:03 00000.txt
-rw-r--r--  1 root root  520885 Mar 27 16:03 00001.jpg
-rw-r--r--  1 root root 3264016 Nov 26  2012 00001.ppm
-rw-r--r--  1 root root     169 Mar 27 16:03 00001.txt
-rw-r--r--  1 root root  400637 Mar 27 16:03 00002.jpg
-rw-r--r--  1 root root 3264016 Nov 26  2012 00002.ppm
-rw-r--r--  1 root root      53 Mar 27 16:03 00002.txt
-rw-r--r--  1 root root  623796 Mar 27 16:03 00003.jpg
-rw-r--r--  1 root root 3264016 Nov 26  2012 00003.ppm
-rw-r--r--  1 root root     177 Mar 27 16:03 00003.txt
-rw-r--r--  1 root root  578250 Mar 27 16:03 00004.jpg
-rw-r--r--  1 root root 3264016 Nov 26  2012 00004.ppm
-rw-r--r--  1 root root     113 Mar 27 16:03 00004.txt
-rw-r--r--  1 root

In [ ]:
#전체 이미지와 test에 사용할 이미지 개수 count
num_test = int(len(paths) * test_percentage)
print(len(paths))
print(num_test)

2113
422


In [ ]:
paths_test = paths[:num_test]
paths_train = paths[num_test:]

In [ ]:
with open('train.txt', 'w') as train_txt:
    for path in paths_train:
        train_txt.write(path)
    print('[train.txt] is created')

[train.txt] is created


In [ ]:
with open('test.txt', 'w') as test_txt:
    for path in paths_test:
        test_txt.write(path)
    print('[test.txt] is created')

[test.txt] is created


In [ ]:
#classes.names file에는 category 넣어주기
with open('classes.names', 'w') as names:
    for category in categories:
        names.write(category)
    print('[classes.names] is created')

[classes.names] is created


In [ ]:
with open('traffic.data', 'w') as data:
    data.write('classes = ' + '1\n')
    data.write('train = ' + 'train.txt' + '\n')
    data.write('valid = ' +  'test.txt' + '\n')
    data.write('names = ' +  'classes.names' + '\n')
    data.write('backup = backup')
    print('[custom_data.data] is created')

[custom_data.data] is created


In [ ]:
!mkdir backup

In [ ]:
# traffic-sign-yolo.cfg 업로드
# darknet

from google.colab import files
files.upload()

Saving traffic-sign-yolo.cfg to traffic-sign-yolo.cfg


{'traffic-sign-yolo.cfg': b'[net]\n# Testing\n#batch=1\n#subdivisions=1\n# Training\nbatch=64\nsubdivisions=16\nheight=416\nwidth=416\nchannels=3\nmomentum=0.9\ndecay=0.0005\nangle=0\nsaturation = 1.5\nexposure = 1.5\nhue=.1\n\nlearning_rate=0.001\nburn_in=1000\nmax_batches = 4000\npolicy=steps\nsteps=3200,3600\nscales=.1,.1\n\n[convolutional]\nbatch_normalize=1\nfilters=32\nsize=3\nstride=1\npad=1\nactivation=leaky\n\n# Downsample\n\n[convolutional]\nbatch_normalize=1\nfilters=64\nsize=3\nstride=2\npad=1\nactivation=leaky\n\n[convolutional]\nbatch_normalize=1\nfilters=32\nsize=1\nstride=1\npad=1\nactivation=leaky\n\n[convolutional]\nbatch_normalize=1\nfilters=64\nsize=3\nstride=1\npad=1\nactivation=leaky\n\n[shortcut]\nfrom=-3\nactivation=linear\n\n# Downsample\n\n[convolutional]\nbatch_normalize=1\nfilters=128\nsize=3\nstride=2\npad=1\nactivation=leaky\n\n[convolutional]\nbatch_normalize=1\nfilters=64\nsize=1\nstride=1\npad=1\nactivation=leaky\n\n[convolutional]\nbatch_normalize=1\nf

In [ ]:
#darknet을 changemode에서 실행할 수 있도록 설정
!chmod +x ./darknet

In [ ]:
# Download Pretrained Convolutional Weights
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-03-27 16:07:48--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  39.6MB/s    in 4.3s    

2021-03-27 16:07:53 (36.0 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
!ls -al

total 3345144
drwxr-xr-x 46 root root     73728 Mar 27 16:07 .
drwxr-xr-x  1 root root      4096 Mar 27 15:44 ..
drwxr-xr-x  2 root root      4096 Jul  9  2013 00
-rw-r--r--  1 root root    361627 Mar 27 16:03 00000.jpg
-rw-r--r--  1 root root   3264016 Nov 26  2012 00000.ppm
-rw-r--r--  1 root root         0 Mar 27 16:03 00000.txt
-rw-r--r--  1 root root    520885 Mar 27 16:03 00001.jpg
-rw-r--r--  1 root root   3264016 Nov 26  2012 00001.ppm
-rw-r--r--  1 root root       169 Mar 27 16:03 00001.txt
-rw-r--r--  1 root root    400637 Mar 27 16:03 00002.jpg
-rw-r--r--  1 root root   3264016 Nov 26  2012 00002.ppm
-rw-r--r--  1 root root        53 Mar 27 16:03 00002.txt
-rw-r--r--  1 root root    623796 Mar 27 16:03 00003.jpg
-rw-r--r--  1 root root   3264016 Nov 26  2012 00003.ppm
-rw-r--r--  1 root root       177 Mar 27 16:03 00003.txt
-rw-r--r--  1 root root    578250 Mar 27 16:03 00004.jpg
-rw-r--r--  1 root root   3264016 Nov 26  2012 00004.ppm
-rw-r--r--  1 root root       113 Mar 2

In [ ]:
# Train The Model
!./darknet detector train traffic.data traffic-sign-yolo.cfg darknet53.conv.74 -dont_show